In [1]:
# loading packages
import glob
import gzip
import tqdm.notebook as tq
import json
import re
import pandas as pd
import numpy as np

# reading and combining all of the files 
## (note: safegraph provided the dataset in multiple .gz and .csv folders)
print("Reading the Files...")
ZIPFILES='*.gz'
filelist = glob.glob(ZIPFILES)

df = pd.DataFrame()
for gzfile in tq.tqdm(filelist):
    if len(df)==0:
        df=pd.read_csv(gzfile)
    else:
        data = pd.read_csv(gzfile)
        df=pd.concat([data,df])
print("File reading completed!")
## only selecting columns of interest
df_select = df[["placekey","parent_placekey","location_name","street_address","city","region","brands","date_range_start",
               "poi_cbg","visitor_home_aggregation","distance_from_home"]]
df_select=df_select.sort_values(by="date_range_start")

## converting elements of columns them to its rightful type (i.e., dictionary for "visitor_home_aggregation" & str for "placekey")
df_select["visitor_home_aggregation"]=[json.loads(i) for i in tq.tqdm(list(df_select["visitor_home_aggregation"]))]
df_select["date_range_start"]=[str((str(i))[:7]) for i in tq.tqdm(df_select.date_range_start)]
df_select = df_select.reset_index()
df_select["placekey"]=[str(i) for i in tq.tqdm(df_select["placekey"])]
df_select


# inspecting all the stores and removing POIs that have been classified wrongly as grocery stores. 

print("Identifying misclassified grocery stores...")
stores=df_select[["placekey","location_name"]]
stores=stores.drop_duplicates()
## removing punctuation and special characters from the location names of our stores so it can be easier to filter them accordingly
stores["location_name"]=[(re.sub(r'[^\w\s]', '', i)) for i in tq.tqdm(stores["location_name"])]
## lower spacing all of the location names so it can be easy to filter them by location name
stores["location_name"]=[i.lower() for i in tq.tqdm(stores["location_name"])]
stores["location_name"]=[i.split() for i in tq.tqdm(stores["location_name"])]
stores = stores.reset_index()
stores

# removing stores that have been wrongly misclassified as grocery stores
# identifying placekeys of misclassified grocery-stores
black_list=[]
for i in tq.tqdm(range(len(stores))):
    current=stores.location_name[i]
    current_placekey=str(stores.placekey[i])
    
    # terminologies that are clearly not associated with grocery stores. 
    list_of_unwanted=["bar","bars","sandwich","sandwiches","sandwichs","bakery","donut",
                     "donuts","bakerys","bakeries","cafe","cafes","coffee","coffees","pizza","pizzas","bagel","bagels",
                     "furniture","furnitures","hardware","hardwares","dicks","sport","sports",
                     "electronic","electronical","electric","electronics","clothing","clothes","technology",
                     "technological","technical","technology"]
    
    # list of exceptions - if the location name is a combination of unwanted terminologies but contains terminologies that clearly signify some grocery store, they are included. 
    list_of_exceptions=["grocery","groceries","supermarket","supermarkets","market","markets","farm","farmers","farms","produce"]
    
    if current_placekey in black_list:
        continue
    elif set(current)&set(list_of_exceptions): # some supermarkets may be multipurpose (eg contain a grocery-store and a cafe), we want to preserve such cases
        continue
    elif set(current)&set(list_of_unwanted): # remove options which are clearly not supermarkets (eg bars)
        black_list.append(str(current_placekey))
    else:
        continue

print("number of misclassified grocery stores are: ", len(black_list))


## removing our identified and extracted misclassified grocery-stores from our data. 
print("Removing misclassified grocery stores...")

df_select=df_select[~df_select['placekey'].isin(black_list)] 
df_select = df_select.reset_index()
df_select=df_select.sort_values(by="date_range_start")
df_select
## cleaning the dataframe
df_select=df_select.drop(['level_0', 'index'], axis=1)
df_select
print("The total number of grocery store chain names selected is:",len(list(np.unique(np.array(df_select["location_name"])))))
print("The number of grocery store locations selected is:",len(list(np.unique(np.array(df_select["placekey"])))))

print("converting Census-tract level visitation to County-level visitation...")


# scaling-up the visitation data from census tract to county level
## This function converts the census tract to county's
def conv_to_county(dictionary):
    new_dict={}
    in_the_system=[]
    for i in dictionary.keys():
        if str(i[:5]) in in_the_system:
            old_value=int(new_dict[str(i[:5])])
            to_be_added=int(dictionary[str(i)])
            new_value=int(old_value+to_be_added)
            new_dict[str(i[:5])]=new_value
        else:
            new_dict[str(i[:5])]=int(dictionary[i])
            in_the_system.append(str(i[:5]))
    return(new_dict)

## This function compiles the visitations based on county's visitors
def home_county(list_of_tract):
    county_list=[]
    for i in tq.tqdm(range(len(list_of_tract))):
        current=list_of_tract[i]
        new=conv_to_county(current)
        county_list.append(new)
    return(county_list)

df_select["home_county"]=home_county(df_select["visitor_home_aggregation"])


# transforming the dataframe such we can now see visitors to each store from each county  
def detailed_long_df(old_df):
    location_list,placekey_list,date_range_start_list,county_list,count_list=[],[],[],[],[]
    for i in tq.tqdm(range(len(df_select))):
        location_name,placekey,date_range_start=(df_select["location_name"][i]),(df_select["placekey"][i]),(df_select["date_range_start"][i])
        current_home_county=(df_select["home_county"][i])
        for j in current_home_county.keys():
            ## removing visitors from Canada
            try:
                x=int(j) 
            except ValueError:
                j="NULL"
            if j=="NULL":
                count=0
            else:
                county=str(j)
                count=int(current_home_county[j])
            location_list.append(location_name)
            placekey_list.append(placekey)
            date_range_start_list.append(date_range_start)
            county_list.append(county)
            count_list.append(count)
    df_detail = pd.DataFrame(list(zip(location_list,placekey_list,date_range_start_list,county_list,count_list)), columns =['location','placekey','date_range_start','county','count'])
    return(df_detail)
## executing the function above
df_detail=detailed_long_df(df_select)

## Cleaning the data to prepare for aggregation 
df_detail=df_detail[df_detail["county"].str.contains("NULL")==False] # removing Canada visitors
df_detail["type"]=["supermarket" for i in tq.tqdm(range(len(df_detail)))]

# aggregating county_level visitor stats to grocery stores
print("Aggregating the visitation statistics...")
aggregated=(df_detail.groupby(['date_range_start','county','type']).agg(month_count=('count', 'sum')))
aggregated = aggregated.reset_index()
# saving csv file
print("Saving Data...")
aggregated.to_csv("grocery_aggregated_by_county.csv", index=False)
print("Data Saved!")
aggregated

Reading the Files...


  0%|          | 0/3 [00:00<?, ?it/s]

File reading completed!


  0%|          | 0/3355892 [00:00<?, ?it/s]

  0%|          | 0/3355892 [00:00<?, ?it/s]

...

  0%|          | 0/3342299 [00:00<?, ?it/s]

  0%|          | 0/3342299 [00:00<?, ?it/s]

  0%|          | 0/19488393 [00:00<?, ?it/s]

Aggregating the visitation statistics...
Saving Data...
Data Saved!


,date_range_start,county,type,month_count
0,2019-01,01001,supermarket,4312
1,2019-01,01003,supermarket,10360
2,2019-01,01005,supermarket,824
3,2019-01,01007,supermarket,585
4,2019-01,01009,supermarket,3790
...,...,...,...,...
128681,2022-05,56037,supermarket,1237
128682,2022-05,56039,supermarket,464
128683,2022-05,56041,supermarket,617
128684,2022-05,56043,supermarket,195
